In [ ]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city:str):
    print(city)
    return "30 degrees"
agent = Agent(
    name="Assistant Agent",
    instructions="너는도움을 주는 어시스턴트야, 답변할때 필요하면 tool들을 사용해. ",
    tools=[get_weather]
)

# agent가 실시간으로 뭐하는지 업데이트 해준다고..?
stream = Runner.run_streamed(agent,"안녕 잘지냄? 안녕 서울날씨는 어떄?")

async for event in stream.stream_events():
    if event.type == "raw_response_event":
        continue
    elif event.type == "agent_updated_stream_event":
        print("Agent updated to", event.new_agent.name)
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print(event.item.raw_item.to_dict())
        elif event.item.type == "tool_call_output_item":
            print(event.item.output)
        elif event.item.type == "message_output_item":
            print(ItemHelpers.text_message_output(event.item))
    print("=" * 20)


Agent updated to Assistant Agent
{'arguments': '{"city":"서울"}', 'call_id': 'call_JfIQkmQBuaudQoZfU1LPaBU0', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_06f4823b7e7a2dd900698817a6f688819a835555b3bfddd771', 'status': 'completed'}
서울
30 degrees
안녕하세요! 저는 항상 잘 지내고 있어요. 서울의 현재 날씨는 30도예요. 더운 편이니 시원하게 지내세요! 궁금한 점 또 있으시면 말씀해 주세요.


In [3]:
from agents import Agent, Runner, function_tool, SQLiteSession, trace
from agents.extensions.visualization import draw_graph
from pydantic import BaseModel

session = SQLiteSession("user_2", "ai-memory.db")

class Answer(BaseModel):
    answer: str
    background_explanation: str

@function_tool
def get_weather():
    return "30 degrees"

geaography_agent = Agent(
    name="Geo Expert Agent",
    instructions="You are a expert in geography, you answer questions related to them.",
    handoff_description="Use this to answer geography related questions.",
    tools=[
        get_weather,
    ],
    output_type= Answer
)

economics_agent = Agent(
    name="Economics Expert Agent",
    instructions="You are a expert in economics, you answer questions related to them.",
    handoff_description="Use this to answer economics questions.",
)

main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[
        economics_agent,
        geaography_agent,
    ],
)

#  이거쓰려면 https://graphviz.org/download/ 다운해야함 ;;; 환경변수설정도 해야하고..
# draw_graph(main_agent) 

In [4]:
result = await Runner.run(
    main_agent,
    "태국 북부 지방의 주도는 뭐야",
    session=session
)

print(result.last_agent.name)
print(result.final_output)

Main Agent
태국 북부 지방의 대표적인 주(도)는 **치앙마이(Chiang Mai)**와 **치앙라이(Chiang Rai)**가 있습니다.

- **치앙마이 주(Province):**  
  - **주도(중심도시):** 치앙마이 시(Chiang Mai)

- **치앙라이 주(Province):**  
  - **주도(중심도시):** 치앙라이 시(Chiang Rai)

일반적으로 “태국 북부 지방의 주도”라고 하면 **치앙마이 시(Chiang Mai City)**를 가장 많이 떠올립니다.  
만약 특정 주를 물으신 거라면, 주 이름을 알려주시면 더 정확히 답해드릴 수 있어요!


In [36]:
with trace("user_2"):
    result = await Runner.run(
        main_agent,
        "What is the capital of Colombia's northen province.",
        session=session,
    )
    result = await Runner.run(
        main_agent,
        "What is the capital of Cambodia's northen province.",
        session=session,
    )
    result = await Runner.run(
        main_agent,
        "What is the capital of Thailand's northen province.",
        session=session,
    )